# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [12]:
# Your Code
a = list(db.companies.find({'name':'Babelgum'},{'name': 1}))
a

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [37]:
# Your Code
#sort utiliza tupla y no un diccionario como dice la documentación. Sort y limit se montan fuera de la db.
b = list(db.companies.find({'number_of_employees': {'$gte': 5000}}).sort('number_of_employees', 1).limit(20))

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [33]:
# Your Code


c = list(db['companies'].find({'$and': [{'founded_year': {'$gt': '2000'}}, {'founded_year': {'$lte': '2005'}}]},{'name': 1, '_id':0, 'founded_year': 1}))

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [36]:
# Your Code
d = list(db['companies'].find({'$and': [{"ipo.valuation_amount": {'$gte': 100000000}}, {'founded_year': {'$lte': 2010}}]},{'name': 1, '_id':0, 'ipo': 1}))

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [40]:
# Your Code
e = list(db['companies'].find({'$and': [{'number_of_employees': {'$lte': 1000}}, {'founded_year': {'$lte': 2005}}]}).sort('number_of_employees', 1).limit(10))

### 6. All the companies that don't include the `partners` field.

In [44]:
# Your Code

f = list(db['companies'].find({'partners': {'$exists': False}}))

### 7. All the companies that have a null type of value on the `category_code` field.

In [47]:
# Your Code

g = list(db['companies'].find({'category_code' : {'$type': 10}}))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [50]:
# Your Code
h = list(db['companies'].find({'$and': [{'number_of_employees': {'$gt': 100}},{'number_of_employees': {'$lte': 1000}}]}, {'name': 1, '_id': 0, 'number_of_employees':1}))


### 9. Order all the companies by their IPO price in a descending order.

In [60]:
# Your Code
#Limito a 100 para que no devuelva error por consultar muchos datos
i = list(db['companies'].find().sort("ipo.valuation_amount",-1).limit(100))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [55]:
# Your Code
j= list(db['companies'].find().sort('number_of_employees', -1).limit(10))

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [59]:
# Your Code
k = list(db['companies'].find({'founded_month': {'$gte': 7}}).limit(1000))


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [62]:
# Your Code
l = list(db['companies'].find({'$and': [{'founded_year': {'$lt': 2000}},{'acquisition.price_amount': {'$gte': 1000000}}]}))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [67]:
# Your Code
m = list(db['companies'].find({'acquisition.acquired_year': {'$gt':2010}}, {'name': 1, '_id': 0, 'acquisition':1}).sort('acquisition.price_amount', -1))
m

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code